In [11]:
import pymysql
import pandas as pd
from datetime import datetime, timedelta

import os
import sys
import urllib.request
import re
import json

### kakaotrend 전체기간동안의 경제keyword에 대한 검색량 counts정보 dataframe화
#### 2018.01.01 ~

In [2]:
kakaotrends_df = pd.read_csv('./data/kakaotrend.csv', header=7, sep=',', skip_blank_lines = True)
kakaotrends_df = kakaotrends_df.rename({'일': 'date', '경제':'kakao'}, axis = 'columns')
kakaotrends_df = kakaotrends_df[0:147]
kakaotrends_df


,date,kakao
0,2018-01-01,26
1,2018-01-08,24
2,2018-01-15,23
3,2018-01-22,21
4,2018-01-29,26
...,...,...
142,2020-09-21,28
143,2020-09-28,19
144,2020-10-05,21
145,2020-10-12,24


### googletrend 전체기간동안의 경제keyword에 대한 검색량 counts정보 dataframe화
#### 2015.09.01 ~ 

In [9]:
googletrends_df = pd.read_csv('./data/googletrend.csv', header=1, sep=',', skip_blank_lines = True)
googletrends_df = googletrends_df.rename({'주': 'date', '경제: (대한민국)':'google'}, axis = 'columns')

In [12]:
"""
google: week를 sunday to saturday로 설정
--> 날짜를 하루씩 더함
"""
def plus_one_day(googletrends_df):
    date2 = []
    for i in range(len(googletrends_df)):
        after_one_day = datetime.strptime(googletrends_df.iloc[i].date, '%Y-%m-%d').date() + timedelta(days=1)
        after_one_day = after_one_day.strftime("%Y-%m-%d")
        date2.append(after_one_day)
    return date2

googletrends_df['date'] = plus_one_day(googletrends_df)
googletrends_df

,date,google
0,2015-09-07,69
1,2015-09-14,74
2,2015-09-21,69
3,2015-09-28,62
4,2015-10-05,61
...,...,...
264,2020-09-28,63
265,2020-10-05,81
266,2020-10-12,82
267,2020-10-19,93


### naver datalab 전체기간동안의 경제keyword에 대한 검색량 counts정보 dataframe화
#### 2016.01.01 ~

In [4]:
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

start_date = '2016-01-01'
today = datetime.date.today().strftime('%Y-%m-%d')
url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\""+start_date+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"week\",\"keywordGroups\":[{\"groupName\":\"경제\",\"keywords\":[\"경제\"]}]}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    scrapped = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)

In [5]:
jsonResult = json.loads(scrapped)
jsonResult['results'][0]['data']

[{'period': '2015-12-28', 'ratio': 19.65163},
 {'period': '2016-01-04', 'ratio': 26.05846},
 {'period': '2016-01-11', 'ratio': 25.16599},
 {'period': '2016-01-18', 'ratio': 27.38639},
 {'period': '2016-01-25', 'ratio': 24.72622},
 {'period': '2016-02-01', 'ratio': 23.01888},
 {'period': '2016-02-08', 'ratio': 23.62248},
 {'period': '2016-02-15', 'ratio': 25.2824},
 {'period': '2016-02-22', 'ratio': 23.76476},
 {'period': '2016-02-29', 'ratio': 28.85228},
 {'period': '2016-03-07', 'ratio': 31.58144},
 {'period': '2016-03-14', 'ratio': 28.44701},
 {'period': '2016-03-21', 'ratio': 26.88626},
 {'period': '2016-03-28', 'ratio': 26.26972},
 {'period': '2016-04-04', 'ratio': 25.94205},
 {'period': '2016-04-11', 'ratio': 25.57126},
 {'period': '2016-04-18', 'ratio': 26.74829},
 {'period': '2016-04-25', 'ratio': 25.11425},
 {'period': '2016-05-02', 'ratio': 22.536},
 {'period': '2016-05-09', 'ratio': 27.78304},
 {'period': '2016-05-16', 'ratio': 27.96412},
 {'period': '2016-05-23', 'ratio': 29

In [8]:
navertrends_df = pd.DataFrame(jsonResult['results'][0]['data'])
navertrends_df = navertrends_df.rename({'period': 'date', 'ratio':'naver'}, axis = 'columns')
navertrends_df

,date,naver
0,2015-12-28,19.65163
1,2016-01-04,26.05846
2,2016-01-11,25.16599
3,2016-01-18,27.38639
4,2016-01-25,24.72622
...,...,...
248,2020-09-28,25.29102
249,2020-10-05,28.63240
250,2020-10-12,26.85177
251,2020-10-19,27.25273


In [21]:
tmp = pd.merge(googletrends_df, kakaotrends_df, how='left')
portaltrends_df = pd.merge(tmp, navertrends_df, how='left')
portaltrends_df = portaltrends_df.fillna(0)
portaltrends_df

,date,google,kakao,naver
0,2015-09-07,69,0,0.00000
1,2015-09-14,74,0,0.00000
2,2015-09-21,69,0,0.00000
3,2015-09-28,62,0,0.00000
4,2015-10-05,61,0,0.00000
...,...,...,...,...
264,2020-09-28,63,19,25.29102
265,2020-10-05,81,21,28.63240
266,2020-10-12,82,24,26.85177
267,2020-10-19,93,21,27.25273


### DB연동

In [38]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE], cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

In [39]:
cur.execute("show databases")
print(cur.fetchall())
cur.execute("use CLI")

[{'Database': 'information_schema'}, {'Database': 'CLI'}, {'Database': 'MusicBank'}, {'Database': 'mysql'}, {'Database': 'mytestdb'}, {'Database': 'performance_schema'}, {'Database': 'sys'}]


0

### DB table 생성

In [40]:
"""
검색어 트랜드 정보 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS portal_trends_ratio(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    day bigint(2) NOT NULL,
    google double,
    kakao double,
    naver double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)

0

In [41]:
cur.execute("desc portal_trends_ratio")
table = cur.fetchall()
pd.DataFrame(list(table))

,Field,Type,Null,Key,Default,Extra
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,day,bigint(2),NO,,None,
4,google,double,YES,,None,
5,kakao,double,YES,,None,
6,naver,double,YES,,None,


### 경제keyword counting정보 주별로 저장

In [42]:
"""
날짜에서 year, month, day 추출
"""
def extract_year_and_month_day(period):
    year = period.split('-')[0]
    month = period.split('-')[1]
    day = period.split('-')[2]
    return year, month, day


In [43]:
"""
db table에 저장
"""
for i in range(len(portaltrends_df)):
    query = """ 
    Insert into portal_trends_ratio (year, month, day, google, kakao, naver) values (%d, %d, %d, %f, %f, %f) ;
    """
    year, month, day = extract_year_and_month_day(portaltrends_df.iloc[i].date)
    google_cnt = portaltrends_df.iloc[i].google
    kakao_cnt = portaltrends_df.iloc[i].kakao
    naver_cnt = portaltrends_df.iloc[i].naver

    mystring = (query % (int(year), int(month), int(day), float(google_cnt), float(kakao_cnt), float(naver_cnt)))
    print(mystring)
    cur.execute(mystring)

t into portal_trends_ratio (year, month, day, google, kakao, naver) values (2018, 2, 5, 55.000000, 33.000000, 21.156330) ;
    
 
    Insert into portal_trends_ratio (year, month, day, google, kakao, naver) values (2018, 2, 12, 43.000000, 21.000000, 15.482450) ;
    
 
    Insert into portal_trends_ratio (year, month, day, google, kakao, naver) values (2018, 2, 19, 54.000000, 27.000000, 21.466750) ;
    
 
    Insert into portal_trends_ratio (year, month, day, google, kakao, naver) values (2018, 2, 26, 52.000000, 23.000000, 20.125890) ;
    
 
    Insert into portal_trends_ratio (year, month, day, google, kakao, naver) values (2018, 3, 5, 58.000000, 25.000000, 28.554790) ;
    
 
    Insert into portal_trends_ratio (year, month, day, google, kakao, naver) values (2018, 3, 12, 60.000000, 24.000000, 29.688710) ;
    
 
    Insert into portal_trends_ratio (year, month, day, google, kakao, naver) values (2018, 3, 19, 68.000000, 30.000000, 29.404150) ;
    
 
    Insert into portal_trends_r

In [44]:
# 확인
query = """ 
select * from portal_trends_ratio;
"""
cur.execute(query)
tmp = pd.DataFrame(list(cur.fetchall()))
tmp

,id,year,month,day,google,kakao,naver
0,1,2015,9,7,69.0,0.0,0.00000
1,2,2015,9,14,74.0,0.0,0.00000
2,3,2015,9,21,69.0,0.0,0.00000
3,4,2015,9,28,62.0,0.0,0.00000
4,5,2015,10,5,61.0,0.0,0.00000
...,...,...,...,...,...,...,...
264,265,2020,9,28,63.0,19.0,25.29102
265,266,2020,10,5,81.0,21.0,28.63240
266,267,2020,10,12,82.0,24.0,26.85177
267,268,2020,10,19,93.0,21.0,27.25273


In [45]:
conn.commit()
cur.close()
conn.close()